In [5]:
from datetime import datetime 
import nltk
import numpy as np
import os, sys
import pandas as pd
import re

In [11]:
folder = os.path.join(os.getcwd(), 'data')
file = os.path.join(folder, 'IRAhandle_tweets_1.csv')
print(file)
df = pd.read_csv(file)
#df = pd.read_csv('IRAhandle_tweets_1.csv')

/home/thomas/trolls/data/IRAhandle_tweets_1.csv


info:

region: Regio waar de tweet vandaan komt (?)

author: Accountname

language: taal van de tweet

retweet: 0 of 1, dus is het een retweet of niet
account_category: RightTroll', 'NonEnglish', 'Fearmonger', 'LeftTroll', 'Unknown', 'HashtagGamer', 'NewsFeed', 'Commercial'

Updates: aantal twitter berichten gepost?


In [17]:
df[df['language'] == 'Russian']

,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,...,account_type,retweet,account_category,new_june_2018,alt_external_id,tweet_id,article_url,tco1_step1,tco2_step1,tco3_step1
372,2528776985,1488REASONS,Причина #67 Мутко: «Зенит-Арене» для адаптации...,Unknown,Russian,1/19/2017 13:07,1/19/2017 13:07,6311,6313,1806,...,Russian,0,NonEnglish,0,2528776985,822067907547066368,http://twitter.com/2528776985/statuses/8220679...,NaN,NaN,NaN
373,2528776985,1488REASONS,Причина #70 Житель Самары умер в очереди в пол...,Unknown,Russian,1/22/2017 12:37,4/5/2017 7:17,1580,6298,1869,...,Russian,0,NonEnglish,0,2528776985,823147583895994368,http://twitter.com/2528776985/statuses/8231475...,NaN,NaN,NaN
374,2528776985,1488REASONS,Причина #74 Президентский советник предложил о...,Unknown,Russian,1/26/2017 15:39,1/26/2017 15:40,6305,6312,1813,...,Russian,0,NonEnglish,0,2528776985,824643054351085568,http://twitter.com/2528776985/statuses/8246430...,NaN,NaN,NaN
375,2528776985,1488REASONS,Причина #75 Казаков благословили на защиту инт...,Unknown,Russian,1/27/2017 13:23,1/27/2017 13:23,6304,6311,1814,...,Russian,0,NonEnglish,0,2528776985,824971050161209344,http://twitter.com/2528776985/statuses/8249710...,NaN,NaN,NaN
376,2528776985,1488REASONS,Причина #77 В Кстово полицейские сломали женщи...,Unknown,Russian,1/29/2017 11:07,1/29/2017 11:30,6306,6308,1816,...,Russian,0,NonEnglish,0,2528776985,825661520726089728,http://twitter.com/2528776985/statuses/8256615...,NaN,NaN,NaN
377,2528776985,1488REASONS,Повод #1 https://t.co/cYGcqqL96T,Unknown,Russian,10/10/2016 8:46,10/10/2016 8:46,6375,6362,1745,...,Russian,0,NonEnglish,0,2528776985,785401174174334976,http://twitter.com/1488reasons/statuses/785401...,https://twitter.com/1488reasons/status/7854011...,NaN,NaN
378,2528776985,1488REASONS,Повод #2 Reuters: Путин отказался от поездки в...,Unknown,Russian,10/11/2016 9:07,10/11/2016 9:07,6375,6360,1746,...,Russian,0,NonEnglish,0,2528776985,785768736129036288,http://twitter.com/1488reasons/statuses/785768...,NaN,NaN,NaN
379,2528776985,1488REASONS,Причина #5 https://t.co/JgkjdlIMRU,Unknown,Russian,10/18/2016 11:14,10/18/2016 11:14,6371,6352,1748,...,Russian,0,NonEnglish,0,2528776985,788337328163094532,http://twitter.com/1488reasons/statuses/788337...,https://twitter.com/1488reasons/status/7883373...,NaN,NaN
380,2528776985,1488REASONS,Причина #8 Дым от авианосца «Адмирал Кузнецов»...,Unknown,Russian,10/24/2016 9:49,10/25/2016 7:16,6370,6346,1752,...,Russian,0,NonEnglish,0,2528776985,790490272807391232,http://twitter.com/1488reasons/statuses/790490...,NaN,NaN,NaN
381,2528776985,1488REASONS,Причина #9 «Роснефть» попросила в аренду дворе...,Unknown,Russian,10/25/2016 7:14,10/25/2016 7:15,6370,6346,1752,...,Russian,0,NonEnglish,0,2528776985,790813812370833408,http://twitter.com/1488reasons/statuses/790813...,NaN,NaN,NaN


In [9]:
df

,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,...,account_type,retweet,account_category,new_june_2018,alt_external_id,tweet_id,article_url,tco1_step1,tco2_step1,tco3_step1
0,906000000000000000,10_GOP,"""We have a sitting Democrat US Senator on tria...",Unknown,English,10/1/2017 19:58,10/1/2017 19:59,1052,9636,253,...,Right,0,RightTroll,0,905874659358453760,914580356430536707,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/914580356430...,NaN,NaN
1,906000000000000000,10_GOP,Marshawn Lynch arrives to game in anti-Trump s...,Unknown,English,10/1/2017 22:43,10/1/2017 22:43,1054,9637,254,...,Right,0,RightTroll,0,905874659358453760,914621840496189440,http://twitter.com/905874659358453760/statuses...,https://twitter.com/damienwoody/status/9145685...,NaN,NaN
2,906000000000000000,10_GOP,Daughter of fallen Navy Sailor delivers powerf...,Unknown,English,10/1/2017 22:50,10/1/2017 22:51,1054,9637,255,...,Right,1,RightTroll,0,905874659358453760,914623490375979008,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/913231923715...,NaN,NaN
3,906000000000000000,10_GOP,JUST IN: President Trump dedicates Presidents ...,Unknown,English,10/1/2017 23:52,10/1/2017 23:52,1062,9642,256,...,Right,0,RightTroll,0,905874659358453760,914639143690555392,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/914639143690...,NaN,NaN
4,906000000000000000,10_GOP,"19,000 RESPECTING our National Anthem! #StandF...",Unknown,English,10/1/2017 2:13,10/1/2017 2:13,1050,9645,246,...,Right,1,RightTroll,0,905874659358453760,914312219952861184,http://twitter.com/905874659358453760/statuses...,https://twitter.com/realDonaldTrump/status/914...,NaN,NaN
5,906000000000000000,10_GOP,"Dan Bongino: ""Nobody trolls liberals better th...",Unknown,English,10/1/2017 2:47,10/1/2017 2:47,1050,9644,247,...,Right,0,RightTroll,0,905874659358453760,914320835325853696,http://twitter.com/905874659358453760/statuses...,https://twitter.com/FoxNews/status/91423949678...,NaN,NaN
6,906000000000000000,10_GOP,🐝🐝🐝 https://t.co/MorL3AQW0z,Unknown,English,10/1/2017 2:48,10/1/2017 2:48,1050,9644,248,...,Right,1,RightTroll,0,905874659358453760,914321156466933760,http://twitter.com/905874659358453760/statuses...,https://twitter.com/Cernovich/status/914314644...,NaN,NaN
7,906000000000000000,10_GOP,'@SenatorMenendez @CarmenYulinCruz Doesn't mat...,Unknown,English,10/1/2017 2:52,10/1/2017 2:53,1050,9644,249,...,Right,0,RightTroll,0,905874659358453760,914322215537119234,http://twitter.com/905874659358453760/statuses...,NaN,NaN,NaN
8,906000000000000000,10_GOP,"As much as I hate promoting CNN article, here ...",Unknown,English,10/1/2017 3:47,10/1/2017 3:47,1050,9646,250,...,Right,0,RightTroll,0,905874659358453760,914335818503933957,http://twitter.com/905874659358453760/statuses...,http://www.cnn.com/2017/09/27/us/puerto-rico-a...,NaN,NaN
9,906000000000000000,10_GOP,After the 'genocide' remark from San Juan Mayo...,Unknown,English,10/1/2017 3:51,10/1/2017 3:51,1050,9646,251,...,Right,0,RightTroll,0,905874659358453760,914336862730375170,http://twitter.com/905874659358453760/statuses...,NaN,NaN,NaN
